# 7-дәріс: Модельдің күрделілігі, Артық оқыту және онымен күресу әдістері

**Дәріс мақсаттары:**
1.  Сызықтық емес тәуелділіктерді модельдеу тәсілі ретінде полиномдық регрессияны зерттеу.
2.  ML-дің орталық мәселесін тереңінен түсіну: ығысу мен дисперсия арасындағы ымыра (bias-variance tradeoff), және оның көріністері — кем оқыту мен артық оқыту.
3.  Модельдерді диагностикалау құралдарын меңгеру: оқыту қисықтарын және қалдықтарды визуалды талдау.
4.  Модельдің жалпылау қабілетін сенімді бағалау әдісі ретінде кросс-валидацияны меңгеру.
5.  Модельдің күрделілігін бақылау арқылы артық оқытумен күресу әдісі ретінде реттеу (L1 және L2) идеясын түсіну.

## 1. Полиномдық регрессия: Сызықтық жеткіліксіз болғанда

Өткен дәрісте біз сызықтық регрессияны зерттедік. Бірақ егер деректердегі тәуелділік сызықтық болмаса ше? Қарапайым түзу сызық күрделі деректерді жақсы сипаттай алмайды.

In [ ]:
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt
import numpy as np

# Сызықтық емес (квадраттық) деректерді генерациялайық
np.random.seed(0)
m = 100
X_poly_data = 6 * np.random.rand(m, 1) - 3
y_poly_data = 0.5 * X_poly_data**2 + X_poly_data + 2 + np.random.randn(m, 1)

# Оларды қарапайым сызықтық регрессиямен сипаттауға тырысайық
plain_lin_reg = LinearRegression()
plain_lin_reg.fit(X_poly_data, y_poly_data)

plt.scatter(X_poly_data, y_poly_data, alpha=0.7, label='Бастапқы деректер')
plt.plot(X_poly_data, plain_lin_reg.predict(X_poly_data), color='red', linewidth=2, label='Қарапайым сызықтық регрессия')
plt.title('Қарапайым сызықтық регрессияның шектеулері')
plt.xlabel('X белгісі')
plt.ylabel('y нысаналы айнымалысы')
plt.legend()
plt.show()

### 1.1. Идея: Модельді белгілер арқылы күрделендіру

Егер біз қолданыстағы белгілерден **жасанды түрде жаңа белгілер жасасақ**, сызықтық модельді қисықты сипаттауға мәжбүрлей аламыз. Бұл тәсіл **Полиномдық регрессия** деп аталады.

Біз модельді "алдап", оған тек бастапқы $x_1$ белгісін ғана емес, сонымен қатар оның дәрежелерін ($x_1^2, x_1^3, ...$) де береміз. $x_1$ және $x_2 = x_1^2$ белгілері үшін сызықтық регрессия теңдеуі келесідей болады:

$$ \hat{y} = w_0 + w_1x_1 + w_2x_2 $$

Егер $x_2 = x_1^2$ -ты кері қойсақ, біз мынаны аламыз:

$$ \hat{y} = w_0 + w_1x_1 + w_2x_1^2 $$

Бұл парабола теңдеуі! Сызықтық регрессия моделі үшін ештеңе өзгерген жоқ: ол бұрынғысынша белгілердің *сызықтық* комбинациясын іздейді. Бірақ біз белгілерді түрлендіргеніміздің арқасында қорытынды функция сызықтық емес болады.

### 1.2. Белгілердің өзара әрекеттесуі (Interaction Terms)

Бізде бірнеше бастапқы белгілер болғанда (мысалы, $x_1$ және $x_2$), полиномдық белгілер генераторы олардың дәрежелерін ($x_1^2, x_2^2, ...$) ғана емес, сонымен қатар олардың **көбейтінділерін ($x_1x_2, ...$)** де жасайды. Олардың мағынасы — бір белгінің әсері екіншісінің мәніне байланысты болған кездегі **синергетикалық эффектіні** ұстау.

**Өмірден мысал 1: Жарнамалық кампаниялар**

Instagram-дағы ($x_1$) және көше баннерлеріндегі ($x_2$) жарнама бюджетіне негізделген сатылымды (`y`) болжап жатырмыз деп елестетіңіз.
Қарапайым $y = w_0 + w_1x_1 + w_2x_2$ моделі әрбір арнаның тәуелсіз үлес қосатынын болжайды. Бірақ егер адам Instagram-да жарнаманы көріп, содан кейін көшедегі баннер оған тауарды "есіне түсіріп", эффектіні **күшейтсе** ше? Бұл бірлескен эффект $w_3(x_1x_2)$ қосылғышымен модельденеді.

**Өмірден мысал 2: Ауыл шаруашылығы**

Күн сәулелі күндер санына ($x_1$) және жауын-шашын мөлшеріне ($x_2$) негізделген егін өнімділігін (`y`) болжаймыз. Жаңбырсыз көп күн (қуаңшылық) немесе күнсіз үнемі жаңбыр (шіру) — жаман. Ең жақсы өнім осы факторлардың **тепе-теңдігі** кезінде болады, оны дәл $x_1 \cdot x_2$ өзара әрекеттесу белгісі ұстайды.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures, StandardScaler


# Конвейер (pipeline) құрамыз: алдымен полиномдық белгілерді қосамыз, содан кейін сызықтық регрессияны оқытамыз
poly_reg_model = make_pipeline(PolynomialFeatures(degree=2, include_bias=False), 
                               LinearRegression())

# Модельді оқытамыз
poly_reg_model.fit(X_poly_data, y_poly_data)

# Тегіс қисықты салу үшін деректерді дайындаймыз
X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)
y_plot = poly_reg_model.predict(X_plot)

# Нәтижені визуализациялаймыз
plt.scatter(X_poly_data, y_poly_data, alpha=0.7, label='Бастапқы деректер')
plt.plot(X_plot, y_plot, color='red', linewidth=2, label='Полиномдық регрессия (2 дәреже)')
plt.title('Полиномдық регрессия жұмысының нәтижесі')
plt.xlabel('X белгісі')
plt.ylabel('y нысаналы айнымалысы')
plt.legend()
plt.show()

## 2. Артық оқыту және кем оқыту мәселесі

Полиномдық белгілер — бұл қуатты құрал. Алайда, ол жаңа маңызды сұрақ тудырады: **қай жерде тоқтау керек?** Полиномның қандай дәрежесін таңдау керек? Тым үлкен күрделілік **артық оқытуға** әкеледі.

### 2.1. Ығысу (Bias) және Дисперсия (Variance)

Бұл машиналық оқытудағы орталық мәселе.

*   **Кем оқыту (Underfitting, High Bias):** Модель тым қарапайым және деректердегі негізгі заңдылықтарды ұстай алмайды. Ол **оқыту және жаңа деректерде де нашар сапа** көрсетеді.
*   **Артық оқыту (Overfitting, High Variance):** Модель тым күрделі. Ол тек заңдылықтарды үйреніп қана қоймай, оқыту іріктемесінің кездейсоқ шуын "жаттап алды". Ол **оқыту деректерінде тамаша сапа, бірақ жаңа деректерде өте нашар сапа** көрсетеді.

**Алтын орта:** Бізге заңдылықтарды жақсы жалпылайтын және ескі де, жаңа да деректерде бірдей жақсы жұмыс істейтін модель қажет. Бұл **жақсы жалпылау қабілеті** деп аталады.

Мұны визуалды түрде былай көрсетуге болады:

<table>
  <tr>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-11.png" alt="Жақсы модель" width="400"></td>
    <td><img src="https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-1.png" alt="Артық оқытылған модель" width="400"></td>
  </tr>
</table>

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

plt.figure(figsize=(14, 8))
degrees = [1, 2, 20]
for i, degree in enumerate(degrees):
    ax = plt.subplot(1, len(degrees), i + 1)
    
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_poly_data, y_poly_data)
    
    X_plot = np.linspace(-3, 3, 100).reshape(-1, 1)
    y_plot = model.predict(X_plot)
    
    plt.scatter(X_poly_data, y_poly_data, alpha=0.5)
    plt.plot(X_plot, y_plot, color='red', linewidth=2)
    plt.title(f'Полином дәрежесі = {degree}')
    plt.xlabel('X')
    plt.ylabel('y')
    plt.ylim(0, 10)
    
    if degree == 1:
        plt.text(0.5, 8, "Кем оқыту (High Bias)", fontsize=12)
    if degree == 2:
        plt.text(0, 8, "Оңтайлы модель", fontsize=12)
    if degree == 20:
        plt.text(-2.5, 1, "Артық оқыту (High Variance)", fontsize=12)

plt.tight_layout()
plt.show()

## 3. Модельді диагностикалау және бағалау

**Басты мәселе:** Модельдің артық оқытылғанын қалай түсінуге және оңтайлы күрделілікті қалай табуға болады?

**Негізгі қағида:** Модельдің соңғы сапасын ешқашан ол оқытылған деректерде бағаламау керек. Ол үшін деректерді бөліктерге бөледі.

### 3.1. Деректерді бөлу: Train / Validation / Test

*   **Оқыту іріктемесі (train set, ~60-80%):** Модельді оқыту үшін қолданылады (`w` салмақтарын таңдау).
*   **Валидациялық іріктеме (validation set, ~10-20%):** **Модельді және оның гиперпараметрлерін таңдау** үшін қолданылады (мысалы, оңтайлы полином дәрежесін). Біз *валидациялық* деректерде ең аз қате көрсететін модельді таңдаймыз.
*   **Тест іріктемесі (test set, ~10-20%):** "Тиісуге болмайтын қор". Таңдалған модельдің сапасын соңғы, әділ бағалау үшін **тек бір рет** қолданылады.

Модельдің күрделілігіне байланысты оқыту және тест іріктемелеріндегі қателердің мінез-құлқы — бұл классикалық диагностика әдісі.

![Оқыту қисықтары](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-2.png)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline

X_train, X_val, y_train, y_val = train_test_split(X_poly_data, y_poly_data, test_size=0.3, random_state=10)

train_errors, val_errors = [], []
degrees = range(1, 15)

for degree in degrees:
    model = make_pipeline(PolynomialFeatures(degree), LinearRegression())
    model.fit(X_train, y_train)
    
    y_train_predict = model.predict(X_train)
    y_val_predict = model.predict(X_val)
    
    train_errors.append(np.sqrt(mean_squared_error(y_train, y_train_predict)))
    val_errors.append(np.sqrt(mean_squared_error(y_val, y_val_predict)))

plt.figure(figsize=(8, 5))
plt.plot(degrees, train_errors, 'r-+', linewidth=2, label='Оқытудағы қате (Train)')
plt.plot(degrees, val_errors, 'b-', linewidth=3, label='Валидациядағы қате (Validation)')
plt.legend(loc='upper right', fontsize=14)
plt.xlabel('Модель күрделілігі (полином дәрежесі)', fontsize=14)
plt.ylabel('RMSE', fontsize=14)
plt.title('Оқыту қисықтары', fontsize=16)
plt.grid(True)
plt.tight_layout()
plt.show()

**Графиктен қорытынды (Оқыту қисықтары):**
*   **Оқытудағы** қате күрделілік артқан сайын үнемі төмендейді — модель көрген деректерге барған сайын жақсы бейімделеді.
*   **Валидациядағы** қате алдымен төмендеп, минимумға жетеді (біздің жағдайда 2-3 дәрежеде), содан кейін өсе бастайды. Бұл модельдің артық оқытыла бастайтын нүктесі.

**Біздің мақсатымыз — валидациялық қисықтағы минимумға сәйкес келетін күрделіліктегі модельді табу.**

### 3.2. Қалдықтарды визуалды талдау

Қалдықтар ($y_{True} - y_{pred}$) — бұл біздің модельдің қателері. Идеалды жағдайда, олар заңдылықсыз кездейсоқ шу болуы керек.

**Қалдықтар графигінен не іздейміз:**
1.  **Кездейсоқ таралу:** Нүктелер нөлдік сызықтың айналасында ретсіз шашыраңқы орналасқан.
2.  **Үлгілердің болмауы:** Егер құрылым көрінсе (мысалы, парабола), бұл модель қандай да бір тәуелділікті ұстай алмағанын білдіреді.
3.  **Гомоскедастикалық:** Нүктелердің шашырауы X осінің бойында бірдей болуы керек. Егер шашырау артса (шұңғыл пішіні), бұл гетероскедастикалық деп аталады.

In [ ]:
# Оңтайлы дәрежесі = 2 болатын модельді алайық
import seaborn as sns
optimal_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
optimal_model.fit(X_train, y_train)
y_val_pred = optimal_model.predict(X_val)
residuals = y_val.flatten() - y_val_pred.flatten()

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_val_pred.flatten(), y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Қалдықтар графигі (Residual Plot)')
plt.xlabel('Болжанған мәндер')
plt.ylabel('Қалдықтар (қателер)')
plt.grid(True)
plt.show()

Қалдықтарды талдау тек метрикалар арқылы көрінбейтін мәселелерді анықтауға көмектеседі. Классикалық мысал — **Энскомб квартеті**: бірдей статистикасы және бірдей регрессия сызығы бар, бірақ құрылымы мүлдем әртүрлі төрт деректер жиынтығы.

![Энскомб квартеті](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-3.png)

Тек бірінші деректер жиынтығы үшін (жоғары сол жақта) сызықтық регрессия адекватты модель болып табылады. Қалған жағдайларда қалдықтарды талдау айқын мәселелерді көрсетер еді.

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import seaborn as sns

# Оңтайлы дәрежесі = 2 болатын модельді алайық
optimal_model = make_pipeline(PolynomialFeatures(2), LinearRegression())
optimal_model.fit(X_train, y_train)
y_val_pred = optimal_model.predict(X_val)
residuals = y_val.flatten() - y_val_pred.flatten()

plt.figure(figsize=(10, 6))
sns.scatterplot(x=y_val_pred.flatten(), y=residuals)
plt.axhline(y=0, color='r', linestyle='--')
plt.title('Қалдықтар графигі (Residual Plot)')
plt.xlabel('Болжанған мәндер')
plt.ylabel('Қалдықтар (қателер)')
plt.grid(True)
plt.show()

### 3.3. Кросс-валидация әдісі (Cross-Validation)

Кәдімгі Train/Validation Split-тің мәселесі — бағалаудың кездейсоқ бөлуге қатты тәуелді болуы. **K-Fold Cross-Validation** — бұл сенімдірек бағалау әдісі:

1.  Оқыту іріктемесі K қиылыспайтын бөлікке ("фолдтарға") бөлінеді, мысалы, K=5.
2.  Цикл K рет іске қосылады. Әрбір `i` итерациясында:
    *   `i` блогы **валидациялық** ретінде қолданылады.
    *   Қалған K-1 блок **оқыту** үшін қолданылады.
    *   `i` валидациялық блогында сапа бағаланады.
3.  Алынған K сапа бағалары орташаланады.

#### Жалпы баптау және бағалау процесі

```mermaid
graph LR
    A["Деректер<br/>X және y"] --> B["Оқыту (+ Валидациялық)<br/>деректер жиыны"]
    A --> C["Тест<br/>(кейінге қалдырылған) жиыны"]
    
    subgraph Итеративті баптау циклі
        B --> D["Модельді Train-де<br/>оқыту"]
        D --> F["Validation-да<br/>бағалау"]
        F -->|Бағалау нәтижесі| E["Гиперпараметрлерді<br/>баптау"]
        E --> D
    end
    
    C -->|Қорытынды тексеру| G["Ең жақсы модельді<br/>Test-те бағалау"]
    E -- ең жақсы модель --> G
    G --> H["Модельді<br/>енгізу"]
```

![K-Fold CV](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

**Артықшылықтары:**
*   **Сенімділік:** Бағалау бөлудің кездейсоқтығына аз тәуелді болады.
*   **Деректерді тиімді пайдалану:** Әрбір нысан тесттік рөлінде дәл бір рет болады.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

# X_poly_data және y_poly_data деректері алдыңғы ұяшықтардағыдай
# ноутбукта бұрын анықталуы керек. Мысалдың толықтығы үшін оларды осы жерде қайта анықтайық.
np.random.seed(0)
m = 100
X_poly_data = 6 * np.random.rand(m, 1) - 3
y_poly_data = 0.5 * X_poly_data**2 + X_poly_data + 2 + np.random.randn(m, 1)


# Біздің оңтайлы 2-дәрежелі моделімізді алайық
model_deg_2 = make_pipeline(PolynomialFeatures(2), LinearRegression())

# 10-fold кросс-валидацияны іске қосамыз.
# cv=10 деректердің 10 бөлікке бөлінетінін білдіреді.
# scoring='neg_mean_squared_error' теріс MSE-ді есептейді.
scores = cross_val_score(model_deg_2, X_poly_data, y_poly_data, cv=10, scoring='neg_mean_squared_error')

# Теріс MSE-ді оң RMSE-ге айналдырамыз
rmse_scores = np.sqrt(-scores)

print("10 фолдтың әрқайсысындағы қателер (RMSE):")
print(rmse_scores.round(2))
print("\nКросс-валидациядағы орташа RMSE: {:.2f}".format(rmse_scores.mean()))
print("RMSE стандартты ауытқуы: {:.2f}".format(rmse_scores.std()))

## 4. Реттеу: Артық оқытумен күрес

Артық оқытылған модельдің $w$ салмақтары жиі модуль бойынша өте үлкен болады. Ол тым "жүйкелі", кіріс деректеріндегі кішкене өзгерістерге сезімтал болады.

**Реттеу идеясы:** Шығын функциясына **үлкен салмақтар үшін айыппұл** қосу. Енді модель ымыраға келуге мәжбүр болады: деректерді жақсы сипаттау және сонымен бірге салмақтарды кішкентай ұстау.

$$ L_{new}(w) = L_{old}(w) + \alpha \cdot P(w) = \text{MSE} + \text{айыппұл} $$ 

*   $P(w)$ — **реттеу мүшесі**.
*   $\alpha \ge 0$ — **реттеу коэффициенті**, айыппұлдың күшін бақылайтын гиперпараметр.

### 4.1. L2-реттеу (Ridge Regression, Гребеньдік регрессия)

Айыппұл ретінде салмақтар векторының L2-нормасы (олардың квадраттарының қосындысы) қолданылады.
$$ L_{Ridge}(w) = \sum_{i=1}^{n}(y_i - w^T x_i)^2 + \alpha \sum_{j=1}^{m} w_j^2 $$ 
(бос мүше $w_0$ әдетте реттелмейді)

*   **Әсері:** Барлық салмақтарды нөлге қарай "қысады", бірақ, әдетте, оларды **толығымен нөлге айналдырмайды**.
*   **Қашан пайдалы:** Белгілердің көпшілігі нәтижеге өз үлесін қосқанда жақсы жұмыс істейді.

### 4.2. L1-реттеу (Lasso Regression, Лассо)

Айыппұл ретінде салмақтар векторының L1-нормасы (олардың модульдерінің қосындысы) қолданылады.
$$ L_{Lasso}(w) = \sum_{i=1}^{n}(y_i - w^T x_i)^2 + \alpha \sum_{j=1}^{m} |w_j| $$ 

*   **Әсері:** Кейбір маңыздылығы төмен белгілердің салмақтарын **толығымен нөлге айналдыра алады**.
*   **Негізгі қасиеті:** **Белгілерді автоматты түрде іріктеуді (feature selection)** жүргізеді, модельді қарапайым және интерпретацияланатын етеді.
*   **Қашан пайдалы:** Көптеген белгілер "қоқыс" деп болжаған кезде.

Геометриялық интерпретация L1-реттеудің неліктен салмақтарды нөлге айналдыратынын, ал L2 — жоқ екенін түсінуге көмектеседі. L1 үшін салмақтардың рұқсат етілген мәндерінің аймағы — ромб, ал L2 үшін — шеңбер. Шығын функциясының деңгей сызықтары (эллипстер) ромбқа оның шыңдарының бірінде (коэффициенттердің бірі нөлге тең жерде) тию ықтималдығы жоғары.

![L1 және L2 реттеуінің геометриялық интерпретациясы](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-5.png)

Әсерін визуалды түрде көрейік. 10-дәрежелі полиномдық деректерде (артық оқытуға бейім) 3 модельді оқытайық: қарапайым, Ridge және Lasso.

**Маңызды:** Реттеуді қолданбас бұрын **белгілерді масштабтау қажет** (мысалы, `StandardScaler` көмегімен), әйтпесе айыппұл дұрыс қолданылмайды.

In [ ]:
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
import pandas as pd

# Конвейер құрамыз: масштабтау, полиномдық белгілер, модель
pipe_lr = make_pipeline(StandardScaler(), PolynomialFeatures(degree=10, include_bias=False), LinearRegression())
pipe_ridge = make_pipeline(StandardScaler(), PolynomialFeatures(degree=10, include_bias=False), Ridge(alpha=1))
pipe_lasso = make_pipeline(StandardScaler(), PolynomialFeatures(degree=10, include_bias=False), Lasso(alpha=0.1))

# Оқытамыз
pipe_lr.fit(X_train, y_train)
pipe_ridge.fit(X_train, y_train)
pipe_lasso.fit(X_train, y_train)

# Коэффициенттерді шығарамыз
coeffs_df = pd.DataFrame({
    'Linear Regression': pipe_lr.named_steps['linearregression'].coef_.flatten(),
    'Ridge (alpha=1)': pipe_ridge.named_steps['ridge'].coef_.flatten(),
    'Lasso (alpha=0.1)': pipe_lasso.named_steps['lasso'].coef_.flatten()
})

coeffs_df.plot(kind='bar', figsize=(15, 7))
plt.title('Реттеудің модель коэффициенттеріне әсері (дәреже=10)')
plt.ylabel('Коэффициент мәні')
plt.ylim(-5, 5) # Көрнекілік үшін Y осін шектейік
plt.show()

**Қорытынды:**
*   **Linear Regression:** Коэффициенттердің өте үлкен мәндері бар (мұнда біз оларды көрнекілік үшін шектедік), бұл артық оқытудың айқын белгісі.
*   **Ridge:** Барлық коэффициенттер едәуір кішірейді. Модель тұрақтырақ болды.
*   **Lasso:** Коэффициенттердің көпшілігі нөлге тең болды. Модель ең маңызды белгілерді іріктеді.

### 4.3. ElasticNet және `alpha` гиперпараметрін таңдау

*   **ElasticNet** — бұл L1 және L2 реттеулерінің комбинациясы. Қатты корреляцияланған белгілер топтары болған кезде жақсы жұмыс істейді.
*   **`alpha`-ны таңдау:** Реттеу коэффициенті $\alpha$-ның оңтайлы мәні **кросс-валидация** көмегімен таңдалады. Біз жай ғана бірнеше мәнді (мысалы, `[0.01, 0.1, 1, 10]`) іріктеп, кросс-валидацияда ең жақсы орташа сапаны беретін мәнді таңдаймыз.

## 5. Жетілдірілген тақырыптар: Шығын функциялары және Оңтайландыру әдістері

Бірінші дәрісте біз MSE-мен стандартты шығын функциясы ретінде таныстық. Енді контекстті түсіне отырып, баламаларды қарастырайық.

#### 5.1. Баламалы шығын функциялары

Шығын функциясын таңдау деректердің қасиеттеріне және модельдің мақсаттарына байланысты.

*   **MSE (L2 Loss):** $(y_i - \hat{y}_i)^2$
    *   **Қасиеті:** Үлкен қателер үшін қатты жазалайды. **Шығарындыларға сезімтал**.

*   **MAE (L1 Loss):** $|y_i - \hat{y}_i|$
    *   **Қасиеті:** Қателер үшін сызықты жазалайды. Шығарындыларға **төзімдірек (робасты)**.

*   **Хьюбер шығыны (Huber Loss):** L1 және L2 гибриді.
    *   **Қасиеті:** Кішкентай қателер үшін MSE сияқты, ал үлкен қателер үшін MAE сияқты әрекет етеді. MSE-нің тұрақтылығын және MAE-нің шығарындыларға төзімділігін біріктіреді.

#### 5.2. Оңтайландыру әдістері: Минимумды қалай табуға болады?

*   **Аналитикалық шешім (Қалыпты теңдеу):**
    $$ w = (X^T X)^{-1} X^T y $$
    *   **Артықшылықтары:** Итерациясыз нақты шешім.
    *   **Кемшіліктері:** Белгілер саны көп болғанда есептеу жағынан қымбат ($O(m^3)$). Көптеген модельдерге (мысалы, Lasso) қолданылмайды.

*   **Сандық шешім (Градиенттік түсу):**
    *   **Интуиция:** Шығын функциясының ең тік түсу бағытында (антиградиент) итеративті қадамдар жасау: $w := w - \alpha \cdot ∇L(w)$.

    ![Градиенттік түсу аналогиясы](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-6.png)
    ![Градиенттік түсу аналогиясы 1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-8.png)
    ![Градиенттік түсу аналогиясы 1](https://raw.githubusercontent.com/yuliya-sabirova/ml-course/main/figs/lec7-7.png)

    *   **Артықшылықтары:** Әмбебап және масштабталатын тәсіл. Қазіргі заманғы ML негізі.
    *   **Кемшіліктері:** Жуырланған шешім табады, оқыту жылдамдығын $\alpha$ таңдауды қажет етеді.

## 7-дәріс бойынша қорытынды

Бүгін біз модельдердің жалпылау қабілетімен байланысты негізгі концепцияларды қарастырдық:
1.  **Полиномдық регрессия** сызықтық емес тәуелділіктерді сипаттауға мүмкіндік береді, бірақ **артық оқыту** қаупін тудырады.
2.  Біз **оқыту қисықтары** мен **қалдықтарды талдау** арқылы артық оқыту мен кем оқытуды диагностикалауды үйрендік.
3.  Модельді сенімді бағалау және гиперпараметрлерді таңдау үшін біз **кросс-валидацияны** қолданамыз.
4.  **Реттеу (Ridge, Lasso)** — модельді шамадан тыс күрделілік (үлкен салмақтар) үшін жазалайтын артық оқытумен күресудің қуатты әдісі.
5.  Біз сондай-ақ шығын функциялары мен оңтайландыру әдістеріне оралып, олардың таңдауы модельдің шығарындыларға робастылығы сияқты қасиеттеріне қалай әсер ететінін көрдік.